# LSTM for NLP

In [11]:
import numpy as np
import os
import sys 
import regex as re
from collections import Counter

cwd = os.getcwd()
print(cwd)

/Users/mayazwang/Files/00_Learning/_GenAI_Proj/_Learn_GenAI/traditional-NLP


In [2]:
# read in raw text
sys.path.append('../../data/')
with open('data/anna.txt' , 'r') as f:
    anna = f.read()
print(type(anna))

In [22]:
# clean up line breaks and add space around punctuation (for tokenization)
clean_text=anna.lower().replace("\n", " ") 
clean_text=clean_text.replace("-", " ") 
for x in ",.:;?!$()/_&%*@'`":
    clean_text=clean_text.replace(f"{x}", f" {x} ")
    clean_text=clean_text.replace('"', ' " ') 
text=clean_text.split()

In [24]:
# build vocab
word_counts = Counter(text)
vocab = sorted(word_counts, key=word_counts.get,
reverse = True)
print(vocab[:10])

[',', '.', 'the', '"', 'and', 'to', 'of', 'he', "'", 'a']


In [33]:
# definde tokenizer-encoder and tokenizer-decoder
encoder={v:k for k,v in enumerate(vocab)} 
decoder={k:v for k,v in enumerate(vocab)}
# test
print(encoder['the'])
print(decoder[0],decoder[1],decoder[2])

2
, . the


In [32]:
# apply tokenizer
tokens = [encoder[x] for x in text]
print(text[:20])
print(tokens[:20])

['chapter', '1', 'happy', 'families', 'are', 'all', 'alike', ';', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own', 'way', '.', 'everything', 'was']
[208, 670, 283, 3024, 82, 31, 2461, 35, 202, 690, 365, 38, 690, 10, 234, 147, 166, 1, 149, 12]


In [43]:
# split into train, test sets
len_of_text = len(text)
print(len(tokens))
split_point = int(np.round(len_of_text*0.7))
train_x = tokens[:split_point]
print(len(train_x))
test_x = tokens[split_point:]
print(len(test_x))

440902
308631
132271


In [ ]:
# creat PyTorch dataset 
context_length = 100
from torch.utils.data import Dataset

# 1. Subclass torch.utils.data.Dataset
class my_dataset(Dataset):
    
    # 2. Initialize with customized varargs 
    def __init__(self, tokenized_text: str, max_length: int, stride: int) -> None:
        
        # 3. Create class attributes
        # initialize input_tokens_x and target_tokens_y
        self.input_tokens_x = []
        self.target_tokens_y = []
        # set y as stride number of tokens trailing x 
        for i in range(0, (len(tokenized_text)-max_length), stride):
            x_tmp = tokenized_text[i : (i+max_length)]
            y_tmp = tokenized_text[(i+1) : (i+max_length+1)]
            self.input_tokens_x.append(x_tmp)
            self.target_tokens_y.append(y_tmp)
    
    # 4. Overwrite the __len__() method to return number of rows in the dataset
    def __len__(self) -> int:
        "Returns the number of rows / pairs of x-y sequences in the dataset"
        return len(self.input_tokens_x)
    
    # 5. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        return self.input_tokens_x[idx], self.target_tokens_y[idx]